In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
class HW2(object):
    def __init__(self, path):
        self.path = path
    
    def readFeat(self):
        filenames = os.listdir(self.path)
        for name in filenames:
            if name == 'X_train':
                self.X_train = pd.read_csv(self.path + name)
            elif name == 'Y_train':
                self.Y_train = pd.read_csv(self.path + name)
            else:
                self.X_test = pd.read_csv(self.path + name)
                
        self.X_train_test = pd.concat([self.X_train, self.X_test])
        self.mean = self.X_train_test.mean()
        self.std = self.X_train_test.std()
        self.X_train_norm = (self.X_train - self.mean)/self.std
        self.X_test_norm = (self.X_test - self.mean)/self.std
        return self.X_train_norm, self.Y_train, self.X_test_norm
    
    def calpp(self):
        class_id_0 = self.Y_train[self.Y_train.label==0].index.tolist()
        class_data_0 = self.X_train_norm.loc[class_id_0]
        class_id_1 = self.Y_train[self.Y_train.label==1].index.tolist()
        class_data_1 = self.X_train_norm.loc[class_id_1]
        
        n = class_data_0.shape[1]
        cov_0 = np.zeros((n, n))
        cov_1 = np.zeros((n, n))
        
        mean_0 = class_data_0.mean()
        print(mean_0.shape)
        
        mean_1 = class_data_1.mean()
        print(mean_1.shape)
        
        for i in range(class_data_0.shape[0]):
            cov_0 += np.dot(np.transpose([class_data_0.iloc[i,:] - mean_0]), [(class_data_0.iloc[i,:] - mean_0)]) / class_data_0.shape[0]
        
        for j in range(class_data_1.shape[0]):
            cov_1 += np.dot(np.transpose([class_data_1.iloc[j,:] - mean_1]), [(class_data_1.iloc[j,:] - mean_1)]) / class_data_1.shape[0]
        
        mean_0_r = np.asarray(mean_0).reshape(106, 1)
        mean_1_r = np.asarray(mean_1).reshape(106, 1)
        cov = (cov_0 * class_data_0.shape[0] + cov_1 * class_data_1.shape[1]) / (X_train.shape[0])
        self.W = (np.transpose((mean_0_r - mean_1_r))).dot(np.linalg.inv(cov))
        self.b = (-0.5)*(np.transpose(mean_0_r).dot(np.linalg.inv(cov))).dot(mean_0_r) \
                + 0.5*(np.transpose(mean_1_r).dot(np.linalg.inv(cov)).dot(mean_1_r) \
                + np.log(np.float(class_data_0.shape[0]/np.float(class_data_1.shape[0]))))
        self.W = self.W.reshape(106, )
        return self.W, self.b
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def func(self, X):
        arr = np.empty([X.shape[0], 1])
        for i in range(arr.shape[0]):
            z = X[i, :].dot(self.W) + self.b
            arr[i][0] = self.sigmoid(z)
        self.result = np.clip(arr, 1e-8, 1-(1e-8))
        return self.result

    def predict(self):
        ans = np.zeros((self.result.shape[0], 1))
        for i in range(ans.shape[0]):
            if self.result[i] >= 0.5:
                ans[i][0] = 0
            else:
                ans[i][0] = 1
        return ans

In [3]:
hw2 = HW2('./feature/')
X_train, Y_train, X_test = hw2.readFeat()
W, b = hw2.calpp()
result = hw2.func(np.asarray(X_test))
predict = hw2.predict()

correct = pd.read_csv('correct_answer.csv')
correct['self'] = predict

In [19]:
i = 0
for idx, id, label, self in correct.itertuples():
    if label == self:
        i += 1
rate = i / float(len(correct))
print('correct rate: %f' %rate)    

correct rate: 0.809717
